# Using PEtab

This notebook illustrates how to run model simulations based on [PEtab](https://github.com/petab-dev/petab) problems with AMICI.

PEtab is a format for specifying parameter estimation problems in systems biology. It is based on [SBML](http://sbml.org/) and [TSV](https://en.wikipedia.org/wiki/Tab-separated_values) files. (AMICI also supports PySB-based PEtab problems, that will be covered by PEtab v2). The Python package [pyPESTO](https://pypesto.readthedocs.io/) provides a convenient interface for parameter estimation with PEtab problems and uses AMICI as a backend. However, AMICI can also be used directly to simulate PEtab problems. This is illustrated in this notebook.

In [ ]:
import petab
from amici.importers.petab.v1 import import_petab_problem
from amici.sim.sundials import run_simulation
from amici.sim.sundials.petab.v1 import PetabProblem, simulate_petab
from amici.sim.sundials.plotting import plot_state_trajectories

## Importing a PEtab problem

We use the [Boehm_JProteomeRes2014](https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab/tree/master/Benchmark-Models/Boehm_JProteomeRes2014) example model from the [benchmark collection](https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab):

In [ ]:
model_name = "Boehm_JProteomeRes2014"
# local path or URL to the yaml file for the PEtab problem
petab_yaml = f"https://benchmarking-initiative.github.io/Benchmark-Models-PEtab/tree/Benchmark-Models/{model_name}/{model_name}.yaml"
# load the problem using the PEtab library
petab_problem = petab.Problem.from_yaml(petab_yaml)


Next, we import the model to amici using `import_petab_problem`. `import_petab_problem` has many options to choose between faster importer or more flexible or faster model simulations. We import the model with default settings, and we obtain an AMICI model instance:

In [ ]:
amici_model = import_petab_problem(petab_problem, verbose=False, compile_=True)

That's it. Now, we can use the model to perform simulations.

## Simulating a PEtab problem

For simple simulations, a function `simulate_petab` is available. This function will simulate the model for all conditions specified in the PEtab problem and compute the objective value (and if requested, the gradient). `simulate_petab` is mostly useful for running individual simulations. If large numbers of model simulations are required, there are more efficient means. In particular, for parameter estimation, consider using the optimized objective function provided by [pyPESTO](https://github.com/icb-dcm/pypesto).

We use the `simulate_petab` function to simulate the model at the nominal parameters (i.e., the parameters specified in the PEtab problem in the `nominalValue` column of the parameter table):

In [ ]:
simulate_petab(petab_problem, amici_model=amici_model)

 Parameters can also be directly specified, both scaled and unscaled:

In [ ]:
parameters = {
    x_id: x_val
    for x_id, x_val in zip(petab_problem.x_ids, petab_problem.x_nominal_scaled)
    # Fixed parameters cannot be changed in `simulate_petab`, unless we explicitly pass
    #  a `parameter_mapping` that was generated with `fill_fixed_parameters=False`
    if x_id not in amici_model.get_fixed_parameter_ids()
}
simulate_petab(
    petab_problem,
    amici_model=amici_model,
    problem_parameters=parameters,
    scaled_parameters=True,
)

## Working with PEtab-defined simulation conditions

`simulate_petab` is convenient for quickly simulating PEtab-based problems, but for certain applications it may be too inflexible.
For example, it is not easily possible to obtain model outputs for time points other than the measurement timepoints specified in the PEtab problem. In such a case, the `PetabProblem` class can be used to easily generate AMICI `ExpData` objects representing PEtab-defined simulation conditions:

In [ ]:
app = PetabProblem(petab_problem, amici_model)

# ExpData for all conditions:
app.get_edatas()

# ExpData for a single condition:
edata = app.get_edata("model1_data1")

In [ ]:
rdata = run_simulation(
    amici_model, solver=amici_model.create_solver(), edata=edata
)
rdata

In [ ]:
plot_state_trajectories(rdata)

For further information, check out the [AMICI documentation](https://amici.readthedocs.io/en/latest/).